In [1]:
!ls

ImageGeneration.ipynb	       imagegeneration.ipynb.amltmp
ImageGenerationTraining.ipynb  imagegenerationtraining.ipynb.amltmp
diffusers		       run_accelerate.sh


In [3]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/bkathuri1/code/Users/bkathuri


In [19]:
import os

os.environ['MODEL_NAME'] = f'CompVis/stable-diffusion-v1-2'
os.environ['DATASET_NAME'] = f'kbharat7/DogChestXrayDatasetNew'
os.environ['OUTPUT_DIR'] = f'newmodel'

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
!ls

ImageGeneration.ipynb	       imagegenerationtraining.ipynb.amltmp
ImageGenerationTraining.ipynb  run_accelerate.sh
diffusers		       run_accelerate.sh.amltmp
imagegeneration.ipynb.amltmp


In [8]:
!chmod a+rwx run_accelerate.sh

In [22]:
!./run_accelerate.sh

03/25/2024 02:55:50 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'sample_max_value', 'timestep_spacing', 'variance_type', 'rescale_betas_zero_snr', 'clip_sample', 'prediction_type', 'clip_sample_range', 'thresholding', 'dynamic_thresholding_ratio'} was not found in config. Values will be initialized to default values.
{'latents_std', 'norm_num_groups', 'scaling_factor', 'latents_mean', 'force_upcast'} was not found in config. Values will be initialized to default values.
{'projection_class_embeddings_input_dim', 'class_embeddings_concat', 'addition_embed_type_num_heads', 'upcast_attention', 'encoder_hid_dim', 'time_embedding_dim', 'addition_embed_type', 'addition_time_embed_dim', 'num_class_embeds', 'conv_out_kernel', 'reverse_transformer_layers_per_block', 'use_linear_projection', 'dual_cross_attention', 'resnet_out_scale_factor', 'resnet_skip_time_act', 'encoder_hid_dim_type', 'tran

In [24]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

In [28]:
connect_str = '' # Replace 'your_connection_string' with your actual connection string
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_name = 'imagegeneration' # Replace 'your_container_name' with your actual container name


In [29]:
local_path = "newmodel" # Replace 'your_local_folder' with your folder path

In [30]:
def upload_directory_to_blob(local_directory, container_name):
    """
    Uploads a directory to a specified Azure Blob Storage container.
    """
    container_client = blob_service_client.get_container_client(container_name)
    for root, dirs, files in os.walk(local_directory):
        for name in files:
            file_path = os.path.join(root, name)
            blob_path = os.path.relpath(file_path, local_directory)
            blob_client = container_client.get_blob_client(blob_path)
            with open(file_path, "rb") as data:
                blob_client.upload_blob(data)
                print(f"Uploaded {file_path} to blob storage")

# Execute the upload
upload_directory_to_blob(local_path, container_name)


Uploaded newmodel/model_index.json to blob storage
Uploaded newmodel/feature_extractor/preprocessor_config.json to blob storage
Uploaded newmodel/logs/text2image-fine-tune/events.out.tfevents.1711335355.bkathuri1.15291.0 to blob storage
Uploaded newmodel/logs/text2image-fine-tune/1711335355.5310752/events.out.tfevents.1711335355.bkathuri1.15291.1 to blob storage
Uploaded newmodel/logs/text2image-fine-tune/1711335355.6599011/hparams.yml to blob storage
Uploaded newmodel/safety_checker/config.json to blob storage
Uploaded newmodel/safety_checker/model.safetensors to blob storage
Uploaded newmodel/scheduler/scheduler_config.json to blob storage
Uploaded newmodel/text_encoder/config.json to blob storage
Uploaded newmodel/text_encoder/model.safetensors to blob storage
Uploaded newmodel/tokenizer/merges.txt to blob storage
Uploaded newmodel/tokenizer/special_tokens_map.json to blob storage
Uploaded newmodel/tokenizer/tokenizer_config.json to blob storage
Uploaded newmodel/tokenizer/vocab.jso

In [31]:
# Stable Diffusion V1
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

In [32]:
model_path = "newmodel"

In [33]:
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [01:07<00:00,  9.63s/it]


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.28.0.dev0",
  "_name_or_path": "newmodel",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [37]:
image1 = pipe(prompt="photo of dogxraysmall").images[0]
image1.save("ProductSnapAI_panel_1.png")

100%|██████████| 50/50 [00:01<00:00, 33.44it/s]


In [38]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

In [39]:
# Initialize a connection to the Azure blob storage
connect_str = '' # Replace 'your_connection_string' with your actual connection string
container_name = 'imagegeneration'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

In [45]:
folder_name = 'Large/'

In [46]:
# Your function to generate images
def generate_and_upload_images(num_images):
    for i in range(num_images):
        # Replace 'pipe' with the actual code to generate your image.
        # The following line is just a placeholder for the image generation process.
        image = pipe(prompt="photo of dogxraylarge").images[0]  # This line should be replaced with your image generation code

        # Save the image temporarily. You might need to save it directly if your library supports it.
        file_name = f"large_{i}.png"
        temp_path = os.path.join('Temp', file_name)  # Temporary path in your local environment
        image.save(temp_path)

        # Create a blob client for the image
        blob_client = container_client.get_blob_client(blob=folder_name + file_name)

        # Upload the image to Azure Blob Storage
        with open(temp_path, "rb") as data:
            blob_client.upload_blob(data)

        # Optionally, delete the local file after upload if you want to save space
        os.remove(temp_path)

# Run the image generation and uploading process
generate_and_upload_images(500)

100%|██████████| 50/50 [00:01<00:00, 33.89it/s]
